In [1]:
import numpy as np
import random
import networkx as nx
import matplotlib.pyplot as plt

In [20]:
def busca_local(sol_inicial, matriz_custos, tamanho_tabu=5, iteracoes_max=10000):
    """
    Melhora uma solução pelo método de busca tabu.

    Parâmetros:
        sol_inicial (list[int]): A solução inicial a ser melhorada.
        matriz_custos (list[list[float]]): Matriz de custos associando cada par de elementos.
        tamanho_tabu (int): Tamanho da lista tabu.
        iteracoes_max (int): Número máximo de iterações.

    Retorna:
        melhor_solucao (list[int]): Melhor solução encontrada.
    """
    solucao_atual = sol_inicial[:]
    melhor_solucao = sol_inicial[:]
    melhor_custo = calcular_custo(sol_inicial, matriz_custos)
    lista_tabu = []

    iteracoes = 0

    while iteracoes < iteracoes_max:
        melhorou = False
        melhor_vizinho = None
        melhor_vizinho_custo = float('inf')

        for i in range(len(solucao_atual)):
            for j in range(i + 1, len(solucao_atual)):
                nova_solucao = solucao_atual[:]
                nova_solucao[i], nova_solucao[j] = nova_solucao[j], nova_solucao[i]

                movimento = (solucao_atual[i], solucao_atual[j])

                if movimento not in lista_tabu:
                    custo_vizinho = calcular_custo(nova_solucao, matriz_custos)

                    if custo_vizinho < melhor_vizinho_custo:
                        melhor_vizinho = nova_solucao
                        melhor_vizinho_custo = custo_vizinho
                        melhorou = True

        if melhorou:
            solucao_atual = melhor_vizinho
            if melhor_vizinho_custo < melhor_custo:
                melhor_solucao = melhor_vizinho[:]
                melhor_custo = melhor_vizinho_custo

            lista_tabu.append(movimento)
            if len(lista_tabu) > tamanho_tabu:
                lista_tabu.pop(0)

        iteracoes += 1

    return melhor_solucao

def calcular_custo(solucao, matriz_custos):
    """
    Calcula o custo total de uma solução com base na matriz de custos.
    
    Parâmetros:
        solucao (list[int]): Solução para a qual o custo será calculado.
        matriz_custos (list[list[float]]): Matriz de custos entre os elementos.
    
    Retorna:
        float: O custo total da solução.
    """
    custo_total = 0

    for i in range(len(solucao) - 1):
        custo_total += matriz_custos[solucao[i]][solucao[i+1]]

    custo_total += matriz_custos[solucao[-1]][solucao[0]]  # Assumindo problema circular
    return custo_total


In [21]:
def grasp(matriz_custos, max_iter, alpha):
    """
    Executa o algoritmo GRASP para um problema de otimização.
    
    Parâmetros:
        matriz_custos (list[list[float]]): Matriz de custos associando cada par de elementos.
        max_iter (int): Número máximo de iterações para executar o GRASP.
        alpha (float): Fator de aleatoriedade na construção gulosa.
    
    Retorna:
        melhor_solucao (list[int]): A melhor solução encontrada.
    """
    melhor_solucao = None
    melhor_custo = float('inf')

    for _ in range(max_iter):
        solucao_inicial = construcao_gulosa_randomizada(matriz_custos, alpha) # Constrói uma solução inicial
        solucao = busca_local(solucao_inicial, matriz_custos) # Melhora a solução com busca local
        custo_solucao = calcular_custo(solucao, matriz_custos) # Calcula o custo da solução

        if custo_solucao < melhor_custo: # Atualiza a melhor solução, se necessário
            melhor_solucao = solucao[:]
            melhor_custo = custo_solucao

    return melhor_solucao, melhor_custo

In [22]:
def grasp_reativo(matriz_custos, max_iter, alphas, num_amostras=10):
    """
    Executa o algoritmo GRASP Reativo para um problema de otimização.
    
    Parâmetros:
        matriz_custos (np.array): Matriz de custos associando cada par de elementos.
        max_iter (int): Número máximo de iterações para executar o GRASP.
        alphas (list[float]): Lista de valores possíveis para o parâmetro alpha.
        num_amostras (int): Número de amostras para cada valor de alpha antes de ajustar probabilidades.
    
    Retorna:
        tuple: A melhor solução encontrada e o custo associado.
    """

    num_alphas = len(alphas)
    alpha_scores = np.zeros(num_alphas)
    alpha_usos = np.zeros(num_alphas, dtype=int)
    melhor_solucao = None
    melhor_custo = float('inf')

    for iteracao in range(max_iter):

        if iteracao >= num_alphas * num_amostras: # Selecionar um alpha baseado no desempenho passado (após as amostras iniciais)
            prob_alphas = alpha_scores / alpha_usos
            prob_alphas = prob_alphas / sum(prob_alphas)  # Normalizar para criar uma distribuição de probabilidade
            alpha_index = np.random.choice(range(num_alphas), p=prob_alphas)
        else:
            alpha_index = iteracao % num_alphas # Inicialmente usar cada alpha um número igual de vezes

        alpha = alphas[alpha_index]
        alpha_usos[alpha_index] += 1

        solucao_inicial = construcao_gulosa_randomizada(matriz_custos, alpha)
        solucao_melhorada = busca_local(solucao_inicial, matriz_custos)
        custo_solucao = calcular_custo(solucao_melhorada, matriz_custos)

        if custo_solucao < melhor_custo:
            melhor_solucao = solucao_melhorada
            melhor_custo = custo_solucao

        alpha_scores[alpha_index] += 1 / (custo_solucao + 1) # Atualizar a pontuação de alpha

    return melhor_solucao, melhor_custo


In [23]:
def algoritmo_guloso(matriz_custos):
    n = len(matriz_custos)
    visitados = [False] * n
    melhor_solucao = []
    melhor_custo = 0
    atual = 0
    visitados[atual] = True
    melhor_solucao.append(atual)

    for _ in range(n - 1):
        next_index = -1
        min_custo = float('inf')

        for j in range(n):
            if not visitados[j] and matriz_custos[atual][j] < min_custo:
                min_custo = matriz_custos[atual][j]
                next_index = j

        visitados[next_index] = True
        melhor_solucao.append(next_index)
        melhor_custo += min_custo
        atual = next_index


    custo = calcular_custo(melhor_solucao, matriz_custos)

    return melhor_solucao, custo


In [24]:
'''Esses são os números para cada cidade, ou seja na linha 0 coluna 1 temos a distância de Goiânia para Anápolis'''

cidade_num = {
    'Goiânia' :  0,
    'Anápolis' : 1,
    'Luziânia' : 2,
    'Bela Vista de Goiás' : 3,
    'Planaltina' : 4,
    'Trindade' : 5,
    'Pirenópolis': 6,
    'Brasília' : 7,
    'Goiás velho' : 8,
    'Senador Canedo' : 9,
    'Catalão' : 10,
    'Goianésia' : 11,
    'Caldas Novas' : 12,
    'Jataí' : 13,
    'Formosa' : 14,
    'Vianópolis' : 15,
    'Itumbiara' : 16,
    'Mineiros' : 17,
    'Goianira' : 18,
    'Porangatu' : 19
}

In [25]:
'''A matriz contém as distâncias reais entre as cidades, o número respectivo a linha é a cidade de saída e o respectivo a coluna é a cidade de chegada. Esse grafo em si é bidirecional, considerando as distâncias de x a y como de y a x.'''

matriz = [
    [0   , 59.6, 198 , 51.0, 260 , 26.7, 124 , 207 , 142 , 21.1, 260 , 175 , 169 , 321 , 281 , 93.8, 208 , 426 , 29.9, 409],
    [59.3, 0   , 159 , 105 , 207 , 82.1, 63.1, 154 , 158 , 69  , 273 , 139 , 220 , 380 , 228 , 79.0, 262 , 489 , 70.2, 377],
    [199 , 159 , 0   , 187 , 115 , 225 , 162 , 60.6, 317 , 193 , 259 , 244 , 249 , 518 , 136 , 104 , 356 , 623 , 228 , 448],
    [51.0, 104 , 187 , 0   , 304 , 77.1, 173 , 252 , 193 , 38.9, 211 , 225 , 121 , 373 , 325 , 82.8, 185 , 477 , 80.3, 458],
    [259 , 205 , 114 , 300 , 0   , 282 , 200 , 57.5, 367 , 261 , 348 , 276 , 363 , 580 , 38.7, 218 , 462 , 684 , 283 , 430],
    [26.9, 81.2, 225 , 77.0, 281 , 0   , 145 , 229 , 137 , 46.8, 286 , 191 , 195 , 315 , 302 , 121 , 232 , 419 , 24.5, 422],
    [124 , 62.8, 160 , 169 , 199 , 146 , 0   , 150 , 169 , 133 , 333 , 82.6, 287 , 444 , 220 , 139 , 326 , 548 , 146 , 345],
    [208 , 154 , 62.0, 249 , 57.3, 233 , 159 , 0   , 314 , 218 , 316 , 206 , 311 , 528 , 78.6, 166 , 413 , 633 , 234 , 416],
    [142 , 159 , 327 , 193 , 365 , 136 , 169 , 317 , 0   , 162 , 402 , 179 , 314 , 379 , 387 , 235 , 348 , 463 , 113 , 382],
    [23.6, 58.0, 193 , 40.3, 269 , 49.8, 133 , 217 , 165 , 0   , 249 , 190 , 159 , 345 , 290 , 84.8, 222 , 450 , 53.0, 423],
    [262 , 269 , 258 , 213 , 347 , 288 , 334 , 315 , 404 , 250 , 0   , 405 , 119 , 497 , 359 , 194 , 206 , 602 , 291 , 643],
    [176 , 139 , 242 , 225 , 275 , 187 , 82.2, 198 , 179 , 189 , 409 , 0   , 344 , 493 , 290 , 215 , 382 , 597 , 164 , 243],
    [169 , 219 , 249 , 120 , 362 , 195 , 285 , 309 , 311 , 157 , 119 , 356 , 0   , 402 , 374 , 145 , 130 , 506 , 193 , 576],
    [321 , 380 , 518 , 372 , 579 , 315 , 449 , 527 , 379 , 343 , 498 , 494 , 404 , 0   , 601 , 414 , 289 , 108 , 338 , 727],
    [281 , 226 , 135 , 322 , 39.0, 304 , 221 , 79.7, 384 , 282 , 355 , 291 , 373 , 601 , 0   , 239 , 483 , 706 , 305 , 445],
    [95.2, 77.4, 105 , 82.7, 218 , 121 , 144 , 164 , 233 , 89.3, 198 , 215 , 145 , 414 , 240 , 0   , 252 , 519 , 124 , 453],
    [209 , 262 , 356 , 186 , 462 , 235 , 331 , 410 , 348 , 220 , 206 , 382 , 130 , 289 , 483 , 252 , 0   , 394 , 236 , 616],
    [425 , 489 , 623 , 477 , 684 , 420 , 553 , 631 , 431 , 448 , 602 , 598 , 508 , 108 , 705 , 519 , 393 , 0   , 443 , 831],
    [29.1, 70.2, 227 , 82.7, 279 , 22.8, 146 , 227 , 114 , 48.9, 292 , 167 , 201 , 336 , 300 , 123 , 235 , 440 , 0   , 401],
    [410 , 377 , 447 , 459 , 430 , 422 , 345 , 409 , 382 , 423 , 643 , 243 , 577 , 726 , 445 , 453 , 616 , 831 , 399 , 0]
]


In [27]:
greedy_solution_guloso, cost_guloso = algoritmo_guloso(matriz)  #implementando guloso
# Implementando reativo
max_iterations = 100
alphas = [0.1, 0.2, 0.3, 0.35, 0.4, 0.45]
best_solution_grasp_reactive, best_cost_grasp_reactive = grasp_reativo(matriz, max_iterations, alphas)

In [28]:
print("Solução gulosa:", greedy_solution_guloso)
print("Custo solução gulosa:", cost_guloso)

print('')

print("Solução GRASP REATIVO:", best_solution_grasp_reactive)
print("Custo GRASP REATIVO:", best_cost_grasp_reactive)

Solução gulosa: [0, 9, 3, 5, 18, 1, 6, 11, 8, 15, 2, 7, 4, 14, 10, 12, 16, 13, 17, 19]
Custo solução gulosa: 3296.5

Solução GRASP:  [0, 5, 18, 8, 19, 11, 6, 1, 9, 3, 15, 2, 7, 4, 14, 10, 12, 16, 13, 17]
Custo GRASP:  2814.8999999999996

Solução GRASP REATIVO: [18, 5, 0, 3, 9, 1, 6, 11, 19, 14, 4, 7, 2, 15, 10, 12, 16, 13, 17, 8]
Custo GRASP REATIVO: 2681.8
